# Assignment 4: Pipelines and Hyperparameter Tuning (32 total marks)
### Due: November 22 at 11:59pm

### Name: 

### In this assignment, you will be putting together everything you have learned so far. You will need to find your own dataset, do all the appropriate preprocessing, test different supervised learning models and evaluate the results. More details for each step can be found below.

### You will also be asked to describe the process by which you came up with the code. More details can be found below. Please cite any websites or AI tools that you used to help you with this assignment.

## Import Libraries

In [1]:
import numpy as np
import pandas as pd

## Step 1: Data Input (4 marks)

Import the dataset you will be using. You can download the dataset onto your computer and read it in using pandas, or download it directly from the website. Answer the questions below about the dataset you selected. 

To find a dataset, you can use the resources listed in the notes. The dataset can be numerical, categorical, text-based or mixed. If you want help finding a particular dataset related to your interests, please email the instructor.

**You cannot use a dataset that was used for a previous assignment or in class**

In [10]:
# Import dataset (1 mark)

#Using pip install method from UC Irvine
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
mushroom = fetch_ucirepo(id=73) 
  
# data (as pandas dataframes) 
X = mushroom.data.features 
y = mushroom.data.targets 

### Questions (3 marks)

1. (1 mark) What is the source of your dataset?   
1. (1 mark) Why did you pick this particular dataset?    
1. (1 mark) Was there anything challenging about finding a dataset that you wanted to use?
###
1. This dataset comes from the UC Irvine Machine Learning Repository: Mushroom. (1987). UCI Machine Learning Repository. https://doi.org/10.24432/C5959T.

2. I sometimes go mushroom foraging in the fall for chanterelles, so seemed fitting to use ML to see if there's a way to identify other mushrooms I come across.

3. Not specifically since I knew the UC Irvine ML Repository was a good place to find datasets, I just needed to find one I thought was interesting.

## Step 2: Data Processing (5 marks)

The next step is to process your data. Implement the following steps as needed.

In [11]:
# Clean data (if needed)
#X.describe()
#X.isnull().sum()
X.drop('stalk-root', axis=1)
X.describe()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,6,4,10,2,9,2,2,2,12,2,...,4,9,9,1,4,3,5,9,6,7
top,x,y,n,f,n,f,c,b,b,t,...,s,w,w,p,w,o,p,w,v,d
freq,3656,3244,2284,4748,3528,7914,6812,5612,1728,4608,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [12]:
# Implement preprocessing steps. Remember to use ColumnTransformer if more than one preprocessing method is needed
X = pd.get_dummies(X)
y = pd.get_dummies(y, drop_first=True)

### Questions (2 marks)

1. (1 mark) Were there any missing/null values in your dataset? If yes, how did you replace them and why? If no, describe how you would've replaced them and why.
2. (1 mark) What type of data do you have? What preprocessing methods would you have to apply based on your data types?
###
1. There were 2480 null values for the column "stalk-root". Since there are 8124 values, this column was dropped since over 30% were null, and replacing with other values doesn't make sense for this dataset (since it is describing mushrooms).
2. This is categorical data, so I used one-hot encoding to get values for the model.

## Step 3: Implement Machine Learning Model (11 marks)

In this section, you will implement three different supervised learning models (one linear and two non-linear) of your choice. You will use a pipeline to help you decide which model and hyperparameters work best. It is up to you to select what models to use and what hyperparameters to test. You can use the class examples for guidance. You must print out the best model parameters and results after the grid search.

In [13]:
# Implement pipeline and grid search here. Can add more code blocks if necessary
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, accuracy_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


# Random Forest Pipeline
rf_pipeline = Pipeline([
    ('classifier', RandomForestClassifier())
])

# Logistic Regression Pipeline
lr_pipeline = Pipeline([
    ('classifier', LogisticRegression())
])

# SVM Pipeline
svm_pipeline = Pipeline([
    ('classifier', SVC())
])



# Define parameter grids
param_grid_rf = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [5, 10, 15]
}

param_grid_lr = {
    'classifier__C': [0.1, 1, 10],
    'classifier__penalty': ['l1'], 
    'classifier__solver': ['liblinear', 'saga']
}

param_grid_svm = {
    'classifier__C': [0.1, 1, 10],
    'classifier__kernel': ['linear', 'rbf']
}

# Define the scoring metrics you want to use
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_score': make_scorer(f1_score)
}

# Create GridSearchCV instances for each algorithm with multiple scoring metrics
grid_search_rf = GridSearchCV(rf_pipeline, param_grid_rf, cv=5, scoring=scoring, refit='f1_score')
grid_search_lr = GridSearchCV(lr_pipeline, param_grid_lr, cv=5, scoring=scoring, refit='f1_score')
grid_search_svm = GridSearchCV(svm_pipeline, param_grid_svm, cv=5, scoring=scoring, refit='f1_score')

# Fit the models
grid_search_rf.fit(X_train, y_train)
grid_search_lr.fit(X_train, y_train)
grid_search_svm.fit(X_train, y_train)

# Get the best parameters based on F1
best_params_rf = grid_search_rf.best_params_
best_params_lr = grid_search_lr.best_params_
best_params_svm = grid_search_svm.best_params_

# Access the results for both scoring metrics
results_rf = grid_search_rf.cv_results_
results_lr = grid_search_lr.cv_results_
results_svm = grid_search_svm.cv_results_

# Print the results for accuracy and F1
print("Random Forest Results:")
print("Accuracy scores:", results_rf['mean_test_accuracy'])
print("F1 scores:", results_rf['mean_test_f1_score'])
print("\nBest Parameters for Random Forest based on F1:", best_params_rf)

print("\nLogistic Regression Results:")
print("Accuracy scores:", results_lr['mean_test_accuracy'])
print("F1 scores:", results_lr['mean_test_f1_score'])
print("\nBest Parameters for Logistic Regression based on F1:", best_params_lr)

print("\nSVM Results:")
print("Accuracy scores:", results_svm['mean_test_accuracy'])
print("F1 scores:", results_svm['mean_test_f1_score'])
print("\nBest Parameters for SVM based on F1:", best_params_svm)


C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\skle

C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was e

Random Forest Results:
Accuracy scores: [0.98982463 0.98982463 0.98966056 1.         1.         1.
 1.         1.         1.        ]
F1 scores: [0.98936149 0.98936119 0.98918775 1.         1.         1.
 1.         1.         1.        ]

Best Parameters for Random Forest based on F1: {'classifier__max_depth': 10, 'classifier__n_estimators': 100}

Logistic Regression Results:
Accuracy scores: [0.99770236 0.99770236 1.         1.         1.         1.        ]
F1 scores: [0.99761818 0.99761818 1.         1.         1.         1.        ]

Best Parameters for Logistic Regression based on F1: {'classifier__C': 1, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}

SVM Results:
Accuracy scores: [0.99934359 0.99770223 1.         1.         1.         1.        ]
F1 scores: [0.99932059 0.99761789 1.         1.         1.         1.        ]

Best Parameters for SVM based on F1: {'classifier__C': 1, 'classifier__kernel': 'linear'}


### Questions (5 marks)

1. (1 mark) Do you need regression or classification models for your dataset?
1. (2 marks) Which models did you select for testing and why?
1. (2 marks) Which model worked the best? Does this make sense based on the theory discussed in the course and the context of your dataset?

###
1. This datasets requires classification models since it is choosing between edible and poisonous
1. I tested Random Forest, Logistic Regression, and SVM as they are the most common classification models that we used in class.
1. SVM worked the best, but all models were extremely close. It does make sense that SVM is the best as it is not an overly large dataset, and SVM works well for datasets of this size.

## Step 4: Validate Model (6 marks)

Use the testing set to calculate the testing accuracy for the best model determined in Step 3.

In [14]:
# Calculate testing accuracy (1 mark)
model = SVC(kernel='linear', C=1, gamma=1).fit(X_train, y_train)
print("Accuracy on training set: {:.2f}".format(model.score(X_train, y_train)))
print("Accuracy on validation set: {:.2f}".format(model.score(X_test, y_test)))

C:\Users\benre\anaconda3\envs\ensf-ml\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy on training set: 1.00
Accuracy on validation set: 1.00



### Questions (5 marks)

1. (1 mark) Which accuracy metric did you choose? 
1. (1 mark) How do these results compare to those in part 3? Did this model generalize well?
1. (3 marks) Based on your results and the context of your dataset, did the best model perform "well enough" to be used out in the real-world? Why or why not? Do you have any suggestions for how you could improve this analysis?

*ANSWER HERE*
1. I used accuracy for the training and testing set.
1. Results are comparable, both scores are incredibly high due to the dataset
1. Based on the results the info can likely be used, but more care needs to go into determining if all different species of mushrooms were accounted for since the result of a false negative could be deadly. I think a larger dataset with more species could lead to better real-world results.

## Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

*DESCRIBE YOUR PROCESS HERE*
1. Mostly sourced the code from the lab session
1. I completed all the steps in order
1. I did not use any AI for this assignment
1. I was initially confused about the very high scores, but then I reviewed the source of the data and it showed that the baseline for this dataset is 100% accuracy.

## Reflection (2 marks)
Include a sentence or two about:
- what you liked or disliked,
- found interesting, confusing, challenging, motivating
while working on this assignment.


*ADD YOUR THOUGHTS HERE*
- I liked being able to use our own dataset, but turns out the one I chose wasn't the best.
- I found the process of using a pipeline interesting as it makes a lot of the tasks easier.